In [9]:
import json
import os
import numpy as np
import regex as re
import pandas as pd

In [10]:
np.random.seed(0)

In [11]:
train_raw = json.load(open('train_ood_codesearchnet.json', 'r'))
print(json.dumps(train_raw[100], indent=4))

{
    "idx": 101,
    "doc": "Extract the loss categories and the tags of the exposure . Use it as / extract / exposure_metadata",
    "code": "def extract_exposure_metadata ( dstore , what ) : dic = { } dic1 , dic2 = dstore [ 'assetcol/tagcol' ] . __toh5__ ( ) dic . update ( dic1 ) dic . update ( dic2 ) if 'asset_risk' in dstore : dic [ 'multi_risk' ] = sorted ( set ( dstore [ 'asset_risk' ] . dtype . names ) - set ( dstore [ 'assetcol/array' ] . dtype . names ) ) names = [ name for name in dstore [ 'assetcol/array' ] . dtype . names if name . startswith ( ( 'value-' , 'number' , 'occupants_' ) ) and not name . endswith ( '_None' ) ] return ArrayWrapper ( numpy . array ( names ) , dic )",
    "raw": "def extract_exposure_metadata(dstore, what):\n    \"\"\"\n    Extract the loss categories and the tags of the exposure.\n    Use it as /extract/exposure_metadata\n    \"\"\"\n    dic = {}\n    dic1, dic2 = dstore['assetcol/tagcol'].__toh5__()\n    dic.update(dic1)\n    dic.update(dic2)\n 

In [12]:
from collections import Counter
from tqdm import tqdm
from bpe import BpeVocabulary

code_vocab_counter = Counter()
text_vocab_counter = Counter()

train_id = json.load(open('../python/train_codesearchnet_0.json', 'r'))

for row in tqdm(train_id):
    code = row['code'].split()
    text = row['doc'].split()
    code_vocab_counter.update(code)
    text_vocab_counter.update(text)

code_bpe_vocab = BpeVocabulary(vocab_size=10000, pct_bpe=0.5)
text_bpe_vocab = BpeVocabulary(vocab_size=10000, pct_bpe=0.5)

100%|██████████| 251820/251820 [00:04<00:00, 56697.38it/s]


In [13]:
code_bpe_vocab.fit(code_vocab_counter)
text_bpe_vocab.fit(text_vocab_counter)

100%|██████████| 1067300/1067300 [01:10<00:00, 15211.79it/s]


5000
5000


100%|██████████| 72846/72846 [00:02<00:00, 25784.87it/s]

5000
5000


In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')

from langdetect import detect


In [15]:
# tokenize
def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

# check if text contains special tokens like urls, emails, etc.
def contains_special_token(text):
    doc = nlp(text)
    for token in doc:
        if token.like_url or token.like_email:
            return True
    return False

In [17]:
train = []

for i, row in enumerate(tqdm(train_raw)):
    train_dict = {}
    train_dict['id'] = 'train-python-ood-{}'.format(row['idx'])
    train_dict['code'] = row['code']
    train_dict['text'] = row['doc']
    train_dict['label'] = row['label']
    train_dict['raw'] = row['raw']

    if len(tokenize(train_dict['text'])) <= 3 or contains_special_token(train_dict['text']):
        continue

    try:
        lang = detect(train_dict['text'])
        if lang != 'en':
            continue
    except:
        continue

    train_dict['code_bpe'] = ' '.join(code_bpe_vocab.tokenize(row['code'].split()))
    train_dict['text_bpe'] = ' '.join(text_bpe_vocab.tokenize(row['doc'].split()))

    train.append(train_dict)

  0%|          | 9/160358 [00:00<1:00:04, 44.49it/s]

100%|██████████| 160358/160358 [42:10<00:00, 63.36it/s] 


In [18]:
print(len(train))

136804


In [19]:
test_raw = json.load(open('valid_ood_codesearchnet.json', 'r'))
print(json.dumps(test_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Extracts video ID from URL .",
    "code": "def get_vid_from_url ( url ) : return match1 ( url , r'youtu\\.be/([^?/]+)' ) or match1 ( url , r'youtube\\.com/embed/([^/?]+)' ) or match1 ( url , r'youtube\\.com/v/([^/?]+)' ) or match1 ( url , r'youtube\\.com/watch/([^/?]+)' ) or parse_query_param ( url , 'v' ) or parse_query_param ( parse_query_param ( url , 'u' ) , 'v' )",
    "raw": "def get_vid_from_url(url):\n        \"\"\"Extracts video ID from URL.\n        \"\"\"\n        return match1(url, r'youtu\\.be/([^?/]+)') or \\\n          match1(url, r'youtube\\.com/embed/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/v/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/watch/([^/?]+)') or \\\n          parse_query_param(url, 'v') or \\\n          parse_query_param(parse_query_param(url, 'u'), 'v')",
    "label": 1
}


In [20]:
test = []

for row in test_raw:
    test_dict = {}
    test_dict['id'] = 'test-python-ood-{}'.format(row['idx'])
    test_dict['code'] = row['code']
    test_dict['text'] = row['doc']
    test_dict['label'] = row['label']
    test_dict['raw'] = row['raw']
    test_dict['code_bpe'] = ' '.join(code_bpe_vocab.tokenize(row['code'].split()))
    test_dict['text_bpe'] = ' '.join(text_bpe_vocab.tokenize(row['doc'].split()))

    if len(tokenize(train_dict['text'])) <= 3 or contains_special_token(train_dict['text']):
        continue

    try:
        lang = detect(train_dict['text'])
        if lang != 'en':
            continue
    except:
        continue

    test.append(test_dict)

In [21]:
np.random.shuffle(train)
np.random.shuffle(test)

valid_size = int(len(train) * 0.1)
valid = train[:valid_size]
train = train[valid_size:]

In [22]:
print(len(train))
print(len(valid))
print(len(test))

123124
13680
7258


In [23]:
with open('data_train.json', 'w') as f:
    json.dump(train, f, indent=2)

with open('data_valid.json', 'w') as f:
    json.dump(valid, f, indent=2)

with open('data_test.json', 'w') as f:
    json.dump(test, f, indent=2)


In [24]:
print(json.dumps(valid[:2], indent=4))

[
    {
        "id": "train-python-ood-7657",
        "code": "def sg_hinge ( tensor , opt ) : assert opt . target is not None , 'target is mandatory.' # default margin opt += tf . sg_opt ( margin = 1 ) # reshape target shape = tensor . get_shape ( ) . as_list ( ) broadcast_shape = [ - 1 ] + [ 1 ] * ( len ( shape ) - 2 ) + [ shape [ - 1 ] ] target = tf . cast ( tf . reshape ( opt . target , broadcast_shape ) , tf . sg_floatx ) # hinge loss out = tf . identity ( tf . maximum ( opt . margin - target * tensor , 0 ) , 'hinge' ) # add summary tf . sg_summary_loss ( out , name = opt . name ) return out",
        "text": "r Returns hinge loss between tensor and target . Args : tensor : A Tensor . opt : target : A Tensor . Labels . margin : An int . Maximum margin . Default is 1 . name : A string . A name to display in the tensor board web UI . Returns : A Tensor . For example tensor = [[ 30 10 40 ] [ 13 30 42 ]] target = [[ 0 0 1 ] [ 0 1 0 ]] tensor . sg_hinge ( target = target one_hot = Tru